In [1]:
import pandas as pd 
from glob import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import random
from PIL import Image
import pickle
import geopandas as gpd
from tqdm import tqdm

In [2]:
# Load config file and get orthophoto data path
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
orthophoto_dir = config['data']['orthophotos']

### Read in orthophoto index file

In [3]:
with open(orthophoto_dir + '/metalink_files/complete_index.pkl', 'rb') as f:
    index_file = pickle.load(f)

index_file = gpd.GeoDataFrame(index_file, geometry='polygon_4326')

In [4]:
# set bbox for munich inner city
from shapely.geometry import box
bbox = box(*[11.542597,48.117574,11.624050,48.158986])
city_tiles = index_file[index_file.polygon_4326.intersects(bbox)].tile_name.tolist()
city_tiles = [name.split('.')[0] for name in city_tiles]
city_tiles[:2]

['32692_5335', '32693_5337']

### Get list of all patches

In [5]:
patches = glob(orthophoto_dir + '/patched/*.tif')
len(patches)

30512

### Draw 100 city patches randomly

In [6]:
# Subset the patches list to include patches that include any of the tile names
city_patches_all = [patch for patch in patches for tile in city_tiles if tile in patch]
city_patches = random.sample(city_patches_all, 100)

### Draw 50 other patches randomly

In [7]:
other_patches_all = [patch for patch in patches if patch not in city_patches]
other_patches = random.sample(other_patches_all, 50)

### Visualize Patches

In [8]:
selected_patches = city_patches + other_patches

In [9]:
# fig, axs = plt.subplots(15, 10, figsize=(20, 30))
# for i, file_path in enumerate(selected_patches):
#     # Open image file
#     img = Image.open(file_path)
#     # Convert image data to numpy array
#     img_data = np.array(img)
#     # Calculate subplot position
#     row = i // 10
#     col = i % 10
#     # Display image data on the subplot
#     axs[row, col].imshow(img_data)
#     axs[row, col].axis('off')  # Hide axes

# plt.tight_layout()
# plt.show()

### Transfer selected patches to folder

In [10]:
import shutil

for patch in tqdm(selected_patches):
    name = patch.split('/')[-1].split('.')[0]
    dest = orthophoto_dir + '/labeling_subset/' + name + '.tif'
    shutil.copyfile(patch, dest)


100%|██████████| 150/150 [00:04<00:00, 30.27it/s]
